In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import sys
from pathlib import Path
import anndata as ad
import scanpy as sc
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import seaborn as sns
from matplotlib import pyplot as plt
sys.path.insert(0, "/home/ylu/project/spateo-release_new")
import spateo_dev as st
sys.path.insert(0, "/home/ylu/project")
from utils import *
import time

os.system('/usr/bin/Xvfb :99 -screen 0 1024x768x24 &')
os.environ['DISPLAY'] = ':99'

%load_ext autoreload
%autoreload 2

2024-07-15 20:18:49.885000: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 20:18:50.031811: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.3/lib64
2024-07-15 20:18:50.031845: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-07-15 20:18:50.742074: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot ope

In [2]:
data_folder = "./results/split_slices_CCF/"
results_folder = "./results/Spateo_new/"
figures_folder = "./results/figures/Spateo_new"
Path(results_folder).mkdir(parents=True, exist_ok=True)

In [3]:
import re
def list_and_sort_files_by_number(directory_path, suffix):
    def extract_number(filename):
        match = re.search(r'\d+', filename)
        return int(match.group()) if match else float('inf')
    try:
        files = [f for f in os.listdir(directory_path) if f.endswith(suffix)]
        sorted_files = sorted(files, key=extract_number)
        return sorted_files
    except FileNotFoundError:
        return f"The directory {directory_path} does not exist."
    except NotADirectoryError:
        return f"The path {directory_path} is not a directory."
    except PermissionError:
        return f"Permission denied to access {directory_path}."

directory_path = data_folder
sorted_files = list_and_sort_files_by_number(directory_path, '.h5ad')

In [4]:
## load the data
from tqdm import tqdm
slices = []
for f in tqdm(sorted_files):
    slices.append(ad.read_h5ad(os.path.join(data_folder, f)))

100%|███████████████████████████| 129/129 [00:22<00:00,  5.85it/s]


In [5]:
spatial_key = "spatial_2D"
anno_key = "cell_type"

In [6]:
## Rotate the data
rotate_key = "spatial_rot"
rotations = np.load("./results/random_rotations.npy", allow_pickle=True)
for i in range(len(slices)):
    slices[i].obsm[rotate_key] = slices[i].obsm[spatial_key][:,:2].copy()
    mean = np.mean(slices[i].obsm[rotate_key], axis=0)
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] - mean
    slices[i].obsm[rotate_key] = slices[i].obsm[rotate_key] @ rotations[i].T + mean
    

In [7]:
spatial_key = "spatial_rot"
key_added = "aligned_spatial"

In [8]:
## Perform the Spateo alignment
sampling_num = 5000
# for i in tqdm(range(76, len(slices)-1)):
for i in tqdm(range(len(slices)-1)):
    slice1, slice2 = slices[i].copy(), slices[i+1].copy()
    sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
    sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
    slice1 = slice1[sampline_idx1,:]
    slice2 = slice2[sampline_idx2,:]
    time_start = time.time()
    align_slices, pis, _  = st.align.morpho_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        device="0",
        mode="SN-S",
        max_iter=200,
        verbose=False,
        partial_robust_level=50,
        beta=1e-5,
        lambdaVF=1e5,
        SVI_mode=True,
        nn_init=True,
        iter_key_added=None,
    )
    time_end = time.time()
    matches = [np.arange(pis[0].shape[0]), np.argmax(pis[0], axis=1)]
    R, t=solve_RT_by_correspondence(align_slices[1].obsm[key_added], align_slices[1].obsm[spatial_key])
    alignment_results = {'sampling_idx1': sampline_idx1, 'sampline_idx2': sampline_idx2, 'R': R, 't': t, 'matches': matches, 'time': time_end - time_start}
    np.save(os.path.join(results_folder, f"slice_{i}_{i+1}_sampling_{sampling_num}.npy"), alignment_results, allow_pickle=True)

  0%|                                     | 0/128 [00:00<?, ?it/s]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%tensor(0.0885, device='cuda:0')
tensor(0.0797, device='cuda:0')
tensor(0.0715, device='cuda:0')
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [2.6165s]


  1%|▏                            | 1/128 [00:02<06:10,  2.92s/it]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%tensor(0.0885, device='cuda:0')
tensor(0.0797, device='cuda:0')
tensor(0.0722, device='cuda:0')
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [0.5471s]


  2%|▍                            | 2/128 [00:03<03:33,  1.70s/it]

...

 98%|██████████████████████████▌| 126/128 [01:38<00:01,  1.68it/s]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%tensor(0.0904, device='cuda:0')
tensor(0.0797, device='cuda:0')
tensor(0.0711, device='cuda:0')
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [0.3700s]


 99%|██████████████████████████▊| 127/128 [01:39<00:00,  1.71it/s]

|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%tensor(0.0893, device='cuda:0')
tensor(0.0788, device='cuda:0')
tensor(0.0692, device='cuda:0')
|-----> [Models alignment based on morpho, mode: SN-S.] in progress: 100.0000%
|-----> [Models alignment based on morpho, mode: SN-S.] finished [0.4023s]


100%|███████████████████████████| 128/128 [01:40<00:00,  1.28it/s]
